# Import modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Create file path

In [2]:
path_to_file = '../txt/shakespeare.txt'

# Encoding

In [3]:
text = open(path_to_file, 'r').read()

In [4]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [5]:
vocab = sorted(set(text))
vocab[:10]

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-']

In [6]:
len(vocab)

84

In [7]:
# for item in enumerate(vocab[:10]):
#     print(item)

In [8]:
char_to_ind = {char:ind for ind, char in enumerate(vocab)}

In [9]:
char_to_ind['H']

33

In [10]:
ind_to_char = np.array(vocab)

In [11]:
ind_to_char[33]

'H'

In [12]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [13]:
encoded_text.shape

(5445609,)

In [14]:
sample = text[:500]

In [15]:
# sample

In [16]:
# encoded_text[:500]

# Creating Batches

In [17]:
text[:500]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

In [18]:
line = 'From fairest creatures we desire increase'
len(line)

41

In [19]:
lines = '''
From fairest creatures we desire increase,\n  
That thereby beauty's rose might never die,\n  
But as the riper should by time decease
'''
len(lines)

134

In [20]:
seq_len = 120

In [21]:
len(text)

5445609

In [22]:
total_num_seq = len(text) // (seq_len+1)
total_num_seq

45005

In [23]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [24]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [25]:
# for item in char_dataset.take(500):
#     print(ind_to_char[item.numpy()])

In [26]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [27]:
def create_seq_targets(seq):
    input_txt = seq[:-1] # Hello my nam
    target_txt = seq[1:] # ello my name
    return input_txt, target_txt

In [28]:
dataset = sequences.map(create_seq_targets)

In [29]:
for input_txt, target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print("".join(ind_to_char[target_txt.numpy()]))    

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [30]:
batch_size = 128

In [31]:
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [32]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int32, tf.int32)>

# Creating the Model

In [33]:
vocab_size = len(vocab)
vocab_size

84

In [34]:
embed_dim = 64

In [35]:
rnn_neurons = 1026

In [36]:
batch_size

128

In [37]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [38]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [40]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    
    model.compile(optimizer='adam', loss=sparse_cat_loss)
    
    return model

In [41]:
model = create_model(vocab_size=vocab_size, embed_dim=embed_dim, rnn_neurons=rnn_neurons, batch_size=batch_size)

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


# Train the model

In [43]:
epochs = 30

In [45]:
# model.fit(dataset, epochs=epochs)

In [46]:
from tensorflow.keras.models import load_model

In [47]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('../../Tensorflow 2/TF_2_Notebooks_and_Data/06-NLP-and-Text-Data/shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [55]:
def generate_text(model, start_seed, gen_size=500, temp=1.0):
    
    num_generate = gen_size
    
    input_eval = [char_to_ind[s] for s in start_seed]
    
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    temperature = temp
    
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions/temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(ind_to_char[predicted_id])
        
    return (start_seed+"".join(text_generated))

In [56]:
print(generate_text(model, "JULIET", gen_size=1000))

JULIETHET. Pale ys upon him. 'Acidimus

  TROILUS following]
    Amen
    To purchase more care. I'll ebbotch the yates:
    'Good at you that, till that be true, sweet boy,
    And out of or being patient as she was deem so high.
  NORFOLK. The wretch that is shall so; but that for that
    When shall we go denows you, uncle, time poison,
     How I do obstare my time. Speak friends! The chamace
     Plaing windows then blows not Alowmemurous within.
    Farewell the paper in the champion thou shalt have it?
  Claud. I hope I have not yet meant my virtue; and much
    rail'd here apace.'
  APEMANTUS. Is it your heart?
  MARIANA. No, no. But hear me,  
  HASTINGS.  [Sirious birding with the KING likewise was whits when fine is to me.
  FIRST MURDERER. Sir, here is she a note.
  CAPUCIUS. He is presumption more in mine.
  VIOLA. The honey, e country for her sixteen, brother of            Knocking with Cyprus. Look you, sir, I am glad to see if we come you.
    There is a bastard bath- i